<a href="https://colab.research.google.com/github/arcursino/tcc_mba/blob/main/Pre_processing_SDV_synthetic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

In [ ]:
df = pd.read_csv('/content/Filtrado_Tryout.csv')
df = df[(df['TIS'].isin(['Current Production', '2', '3', '1', '0', '-1']))]

In [ ]:
df = df[['vin',
 'TSTAMP',
 'cell_id',
 'Vehicle_Type',
 'TRIM_SERIES',
 'Warranty',
 'Front_Cross_Camber_Final_Value',
 'L_F_Camber',
 'CAMBER_FL_INITIAL',
 'R_F_Camber',
 'CAMBER_FR_INITIAL',
 'Rear_Cross_Camber',
 'L_R_Camber',
 'CAMBER_RL_INITIAL',
 'R_R_Camber',
 'CAMBER_RR_INITIAL',
 'Rear_Thrust_Angle_Final_Value',
 'RIDE_HEIGHT_FA_DIFF',
 'RIDE_HEIGHT_FL',
 'RIDE_HEIGHT_FL_INITIAL',
 'RIDE_HEIGHT_FR',
 'RIDE_HEIGHT_RA_DIFF',
 'RIDE_HEIGHT_RL',
 'RIDE_HEIGHT_RR',
 'Steering_Wheel_Angle_Final_Value',
 'Steering_Wheel_Angle_Preset_Value',
 'Front_Sum_Toe_Final_Value',
 'TOE_FA_TOTAL_INITIAL',
 'L_F_Toe',
 'TOE_FL_INITIAL',
 'R_F_Toe',
 'TOE_FR_INITIAL',
 'TOE_RA_TOTAL',
 'TOE_RA_TOTAL_INITIAL',
 'L_R_Toe',
 'TOE_RL_INITIAL',
 'R_R_Toe',
 'TOE_RR_INITIAL',
 'WHEELCENTER_FL',
 'WHEELCENTER_FR',
 'WHEELCENTER_RL',
 'WHEELCENTER_RR',
 'Drive_',
 'Vehicle_',
 'Engine_',
 'Cab_',
 'CabStyle',
 'Engine',
 'RearWheels',
 'SteeringGear',
 'Suspension',
 'Tire',
 'Transmission',
 'Trim_Series_broadcast',
 'Wheel',
 'wheelbase_broadcast',
 'LaneDeparture',
 'MY',
  'TIS'
 ]]

In [ ]:
def transform_categorical_labels_auto(df, unique_threshold=50, exclude_cols=None):
    """
    Automatically identifies potential categorical columns based on dtype and
    unique value count, and transforms their labels into the format
    'columnname_index'.

    Args:
        df (pd.DataFrame): The input DataFrame.
        unique_threshold (int): Maximum number of unique values for a column
                                to be considered categorical. Columns with more
                                unique values than this are likely not
                                simple categories (e.g., IDs, free text).
                                Defaults to 50.
        exclude_cols (list, optional): A list of column names to explicitly
                                     exclude from the automatic transformation,
                                     even if they meet the criteria.
                                     Defaults to None.

    Returns:
        pd.DataFrame: A new DataFrame with transformed categorical labels.
    """
    df_transformed = df.copy() # Work on a copy to not modify the original DataFrame

    exclude_cols = ['TSTAMP', 'TIS']

    if exclude_cols is None:
        exclude_cols = []

    print("--- Starting Automatic Label Transformation ---")

    # 1. Identify potential categorical columns
    # We look for object (strings) and category dtypes
    potential_cols = df_transformed.select_dtypes(include=['object', 'category']).columns

    # Filter these columns:
    # - Exclude columns explicitly listed in exclude_cols
    # - Exclude columns where the number of unique values is above the threshold
    #   (as these are less likely to be simple categories)
    categorical_cols_to_transform = [
        col for col in potential_cols
        if col not in exclude_cols #and df_transformed[col].nunique() <= unique_threshold
    ]

    print(f"Identified {len(categorical_cols_to_transform)} potential categorical columns based on criteria (unique <= {unique_threshold}, excluding {exclude_cols}): {categorical_cols_to_transform}")

    # 2. Transform labels for each identified column
    for col in categorical_cols_to_transform:
        # Ensure the column is category dtype temporarily to get categories/codes
        # .astype('category') is safe even if it's already category
        # .cat accessor gives us access to categorical properties
        try:
            cat_col = df_transformed[col].astype('category')

            # Get the unique categories in the order they are assigned codes (0, 1, 2...)
            # .cat.categories property gives the unique values excluding NaN
            original_categories = cat_col.cat.categories

            # Create the mapping dictionary: {original_value: 'columnname_code'}
            # The index 'i' corresponds to the 0-based code assigned by pandas
            mapping = {
                original_categories[i]: f"{col}_{i}"
                for i in range(len(original_categories))
            }

            # Apply the mapping to the column
            # .map() is suitable here; it applies the mapping and handles NaN correctly
            df_transformed[col] = df_transformed[col].map(mapping)

            #print(f"  Transformed column '{col}'. Mapping used: {mapping}")

        except Exception as e:
            # Handle potential errors during transformation for a specific column
            print(f"  Could not transform column '{col}' due to error: {e}")
            # Optionally, you could remove this column or leave it as is


    print("--- Automatic Label Transformation Complete ---")
    return df_transformed

# --- Example Usage ---

# Apply the function to the DataFrame
# We'll use a threshold of 10 for this small example dataset
# We might want to exclude 'ID' even though it's numerical, or 'Description'
# because it has too many unique values and isn't truly categorical.
# The function's logic automatically excludes 'Weight_kg' (float) and 'Is_Fragile' (bool)
# and 'Description' (unique > 10).
# It will identify 'Color', 'Size', 'City', 'Material', 'Rating'.
# Let's explicitly exclude 'Rating' just to show how exclude_cols works.
df_new = transform_categorical_labels_auto(df, unique_threshold=10, exclude_cols=['Rating'])



In [ ]:
df_new.TSTAMP.unique()

In [ ]:
from pandas.api.types import is_numeric_dtype, is_object_dtype, is_categorical_dtype

def clean_warranty_df(df):
    # 1. Drop columns where more than 80% are null
    thresh = len(df) * 0.2  # minimum non-null count to keep column
    df = df.dropna(axis=1, thresh=thresh)

    # Columns excluding 'Warranty' to fill or drop nulls
    cols = df.columns.drop('Warranty')

    # 2. For Warranty_1 rows, fill null with median or mode
    for col in cols:
        if df[col].isnull().any():
            if is_numeric_dtype(df[col]):
                median_val = df[col].median()
                df[col].fillna(median_val, inplace=True)
            else:
                mode_val = df[col].mode()
                if not mode_val.empty:
                    df[col].fillna(mode_val[0], inplace=True)
                else:
                    # If no mode found, just fill with a placeholder or leave as is
                    df[col].fillna('Unknown', inplace=True)

    cleaned_df = df.copy()

    # Optional: sort index if desired to maintain original order
    cleaned_df = cleaned_df.sort_index()

    return cleaned_df

In [ ]:
processed_df = clean_warranty_df(df_new.copy())

In [ ]:
 #Instalar a biblioteca SDV (se ainda não tiver instalada)
 !pip install sdv


In [ ]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(
    data=processed_df,
    table_name='Alignment')

In [ ]:
metadata.visualize()

In [ ]:
'''metadata.update_column(
    column_name='TSTAMP',
    sdtype='datetime',
    datetime_format='%Y-%m-%d')'''

In [ ]:
metadata.validate()

In [ ]:
metadata.save_to_json(filepath='my_metadata_v1.json')

In [ ]:
metadata = Metadata.load_from_json(filepath='my_metadata_v1.json')

In [ ]:
metadata

In [ ]:
# Display columns with missing values and their count
print("Columns with missing values before filling:")
print(processed_df.isnull().sum()[processed_df.isnull().sum() > 0])

# Identify numerical columns
numerical_cols = processed_df.select_dtypes(include=np.number).columns

# Identify object (transformed categorical) columns
object_cols = processed_df.select_dtypes(include='object').columns

# Fill missing values in numerical columns with the mean
print("\nFilling missing numerical values with mean...")
for col in numerical_cols:
    if processed_df[col].isnull().any():
        mean_value = processed_df[col].mean()
        processed_df[col].fillna(mean_value, inplace=True)
        print(f"  Filled NaN in '{col}' with mean: {mean_value:.2f}")

# Fill missing values in object columns with the mode
print("\nFilling missing object (transformed categorical) values with mode...")
for col in object_cols:
    if processed_df[col].isnull().any():
        # .mode()[0] gets the first mode if there are multiple
        mode_value = processed_df[col].mode()[0]
        processed_df[col].fillna(mode_value, inplace=True)
        print(f"  Filled NaN in '{col}' with mode: '{mode_value}'")

# Verify that there are no more missing values
print("\nColumns with missing values after filling:")
print(processed_df.isnull().sum()[processed_df.isnull().sum() > 0])



In [ ]:
processed_df.isnull().sum()

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(processed_df)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=10000)

In [ ]:
synthetic_data.to_csv('/content/drive/MyDrive/TCC_MBA/alignment_synthetic_data.csv', index=False)

In [ ]:
metadata_1 = metadata.to_dict()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality
from sdv.evaluation.single_table import get_column_plot

# 1. perform basic validity checks
diagnostic = run_diagnostic(processed_df, synthetic_data, metadata)

# 2. measure the statistical similarity
quality_report = evaluate_quality(processed_df, synthetic_data, metadata)



In [ ]:
diagnostic.get_score()

In [ ]:
quality_report.get_score()

In [ ]:
quality_report.get_properties()

In [ ]:
diagnostic.get_properties()

In [ ]:
pio.renderers.default = 'colab'

In [ ]:
fig=quality_report.get_visualization(property_name='Column Shapes')
fig.show()

In [ ]:
fig=quality_report.get_visualization(property_name='Column Pair Trends')
fig.show()

In [ ]:
# Get the properties from the quality_report report
properties = quality_report.get_properties()

# Iterate through properties and get visualization if available
print("Attempting to show visualizations for properties:")
for prop in properties['Property'].unique():
    try:
        print(f"- Showing visualization for property: {prop}")
        fig = quality_report.get_visualization(property_name=prop)
        fig.show()
    except TypeError as e:
        # Catch the TypeError if a property name doesn't have a visualization
        print(f"  Could not get visualization for '{prop}'. Error: {e}")
    except Exception as e:
        # Catch any other potential errors during visualization retrieval
        print(f"  An error occurred while getting visualization for '{prop}': {e}")

In [ ]:
# Get the properties from the diagnostic report
properties = diagnostic.get_properties()

# Iterate through properties and get visualization if available
print("Attempting to show visualizations for properties:")
for prop in properties['Property'].unique():
    try:
        print(f"- Showing visualization for property: {prop}")
        fig = diagnostic.get_visualization(property_name=prop)
        fig.show()
    except TypeError as e:
        # Catch the TypeError if a property name doesn't have a visualization
        print(f"  Could not get visualization for '{prop}'. Error: {e}")
    except Exception as e:
        # Catch any other potential errors during visualization retrieval
        print(f"  An error occurred while getting visualization for '{prop}': {e}")

In [ ]:
properties

In [ ]:
import os
# --- Passo 3: Gerar e Salvar o Relatório Visual (HTML) ---
output_html_file = "relatorio_qualidade_sdv.html"

print(f"\nGerando e salvando o relatório visual em '{output_html_file}'...")

# Chame .get_visualization() e depois use .write_html() para salvar como arquivo HTML
fig = quality_report.get_visualization(property_name='Column Pair Trends')
fig.write_html(output_html_file) # Use write_html instead of to_file

# --- Confirmação ---
if os.path.exists(output_html_file):
    print(f"\nRelatório HTML gerado com sucesso! Abra o arquivo '{output_html_file}' no seu navegador para ver os detalhes.")
else:
    print("\nHouve um problema ao gerar o relatório HTML.")

In [ ]:
#synthetic_data = pd.read_csv('/content/alignment_synthetic_data (1).csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- Passo 1: Selecionar Apenas Colunas Numéricas ---
# As correlações de Pearson/Spearman são tipicamente calculadas para dados numéricos.
# Vamos garantir que estamos trabalhando apenas com esses tipos de colunas.

# Identificar colunas numéricas em ambos os DataFrames para garantir consistência
# Usaremos as colunas numéricas do DataFrame real como referência,
# assumindo que o DataFrame sintético possui as mesmas colunas numéricas.
numerical_cols_real = processed_df.select_dtypes(include=np.number).columns

# Filtrar ambos os DataFrames para incluir apenas as colunas numéricas
processed_df_numerical = processed_df[numerical_cols_real]
synthetic_data_numerical = synthetic_data[numerical_cols_real]

print(f"Comparando correlações para {len(numerical_cols_real)} colunas numéricas.")
print("Colunas numéricas:", list(numerical_cols_real))

# --- Passo 2: Calcular as Matrizes de Correlação ---

# Matriz de Correlação dos Dados Reais
correlation_matrix_real = processed_df_numerical.corr()

# Matriz de Correlação dos Dados Sintéticos
correlation_matrix_synthetic = synthetic_data_numerical.corr()

# --- Passo 3: Visualizar as Matrizes de Correlação Lado a Lado usando Heatmaps ---

# Definir o tamanho da figura. Ajuste conforme a quantidade de colunas.
# Uma figura mais larga é útil para colocar dois heatmaps lado a lado.
fig, axes = plt.subplots(1, 2, figsize=(20, 8)) # 1 linha, 2 colunas

# Heatmap para os Dados Reais
# Removemos 'cbar_kw' daqui
sns.heatmap(
    correlation_matrix_real,
    ax=axes[0],          # Especifica em qual subplot desenhar
    annot=False,         # Defina como True para mostrar valores (pode poluir)
    cmap='coolwarm',     # Esquema de cores
    vmin=-1, vmax=1,     # Garante que as escalas de cor sejam comparáveis (-1 a 1)
    square=True,         # Faz os "quadradinhos" serem quadrados
    # cbar_kw={'label': 'Coeficiente de Correlação'} # REMOVIDO
)
axes[0].set_title('Matriz de Correlação (Dados Reais)', fontsize=14)
axes[0].tick_params(axis='x', rotation=90) # Rotaciona os rótulos do eixo X
axes[0].tick_params(axis='y', rotation=0)  # Garante que os rótulos do eixo Y não girem

# Heatmap para os Dados Sintéticos
# Removemos 'cbar_kw' daqui
sns.heatmap(
    correlation_matrix_synthetic,
    ax=axes[1],          # Especifica em qual subplot desenhar
    annot=False,         # Defina como True para mostrar valores
    cmap='coolwarm',     # Esquema de cores
    vmin=-1, vmax=1,     # Garante que as escalas de cor sejam comparáveis
    square=True,         # Faz os "quadradinhos" serem quadrados
    # cbar_kw={'label': 'Coeficiente de Correlação'} # REMOVIDO
)
axes[1].set_title('Matriz de Correlação (Dados Sintéticos)', fontsize=14)
axes[1].tick_params(axis='x', rotation=90) # Rotaciona os rótulos do eixo X
axes[1].tick_params(axis='y', rotation=0)  # Garante que os rótulos do eixo Y não girem

# Adicionar o label à barra de cor manualmente após a criação do heatmap
# seaborn cria a colorbar axis automaticamente
# Para o primeiro heatmap:
if axes[0].collections: # Check if heatmap was drawn
    cbar0 = axes[0].collections[0].colorbar
    cbar0.set_label('Coeficiente de Correlação')

# Para o segundo heatmap:
if axes[1].collections: # Check if heatmap was drawn
    cbar1 = axes[1].collections[0].colorbar
    cbar1.set_label('Coeficiente de Correlação')


# Ajustar o layout para evitar sobreposição de elementos
plt.tight_layout()

# Exibir os gráficos
plt.show()

# --- Opcional: Calcular a Diferença Absoluta entre as Matrizes ---
# Isso pode te dar uma ideia quantitativa da "distância" entre as correlações.
# Use .abs() para obter o valor absoluto das diferenças.
correlation_difference = (correlation_matrix_real - correlation_matrix_synthetic).abs()

print("\nMatriz de Diferença Absoluta das Correlações:")
# print(correlation_difference) # Descomente para imprimir a matriz de diferenças

# Visualizar a matriz de diferenças absolutas
plt.figure(figsize=(10, 8))
# Removemos 'cbar_kw' daqui também
sns.heatmap(
    correlation_difference,
    annot=False,
    cmap='Reds', # Um colormap que destaca valores maiores
    vmin=0, vmax=1, # Diferença vai de 0 (sem diferença) a 1 (correlação oposta)
    square=True,
    # cbar_kw={'label': 'Diferença Absoluta no Coeficiente de Correlação'} # REMOVIDO
)
plt.title('Diferença Absoluta nas Matrizes de Correlação (Real vs Sintético)', fontsize=16)
plt.tick_params(axis='x', rotation=90)
plt.tick_params(axis='y', rotation=0)

# Adicionar o label à barra de cor para o heatmap de diferença
# Get the current axes and find the colorbar
cbar_diff = plt.gca().collections[0].colorbar
cbar_diff.set_label('Diferença Absoluta no Coeficiente de Correlação')

plt.show()

In [ ]:
correlation_difference

In [ ]:
df.columns

In [ ]:
# Visualizar a relação entre duas colunas específicas (por exemplo, 'L_F_Camber' e 'R_F_Camber')
plt.figure(figsize=(8, 6))
sns.scatterplot(data=processed_df, x='L_F_Toe', y='R_F_Toe', alpha=0.5, label='Real Data')
sns.scatterplot(data=synthetic_data, x='L_F_Toe', y='R_F_Toe', alpha=0.5, label='Synthetic Data')
plt.title('Comparação da Relação entre Left Front Toe e Right Front Toe')
plt.xlabel('L_F_Toe')
plt.ylabel('R_F_Toe')
plt.legend()
plt.show()


In [ ]:
# seaborn.pairplot pode ser usado para visualizar pares de colunas selecionadas
sns.pairplot(processed_df[['L_F_Toe', 'R_F_Toe', 'Steering_Wheel_Angle_Final_Value']])
plt.suptitle('Pair Plot for Selected Toe Columns (Real Data)', y=1.02) # Adiciona título acima
plt.show()

In [ ]:
# seaborn.pairplot pode ser usado para visualizar pares de colunas selecionadas
sns.pairplot(synthetic_data[['L_F_Toe', 'R_F_Toe', 'Steering_Wheel_Angle_Final_Value']])
plt.suptitle('Pair Plot for Selected Toe Columns (Synthetic Data)', y=1.02) # Adiciona título acima
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Passo 1: Selecionar as colunas de interesse ---
# Certifique-se de que as colunas existem em ambos os DataFrames
cols_to_plot = ['L_F_Toe', 'R_F_Toe', 'Steering_Wheel_Angle_Final_Value']

# Verificar se as colunas existem em ambos os DataFrames antes de prosseguir
if not all(col in processed_df.columns for col in cols_to_plot):
    print(f"Erro: Algumas colunas não encontradas no DataFrame real: {[col for col in cols_to_plot if col not in processed_df.columns]}")
elif not all(col in synthetic_data.columns for col in cols_to_plot):
     print(f"Erro: Algumas colunas não encontradas no DataFrame sintético: {[col for col in cols_to_plot if col not in synthetic_data.columns]}")
else:
    # --- Passo 2: Preparar os DataFrames para a comparação ---

    # Selecionar apenas as colunas relevantes de cada DataFrame
    real_subset = processed_df[cols_to_plot].copy()
    synthetic_subset = synthetic_data[cols_to_plot].copy()

    # Adicionar uma coluna para identificar a origem dos dados
    real_subset['Data_Source'] = 'Real'
    synthetic_subset['Data_Source'] = 'Synthetic'

    # Combinar os dois DataFrames
    combined_df = pd.concat([real_subset, synthetic_subset], ignore_index=True)

    print("DataFrames combinados para pair plot com sucesso.")

    # --- Passo 3: Gerar o Pair Plot Comparativo ---

    # Use seaborn.pairplot no DataFrame combinado
    # O argumento 'hue' plota pontos com cores diferentes baseadas na coluna 'Data_Source'
    # O argumento 'diag_kws' permite personalizar os plots diagonais (geralmente histogramas ou kde)
    # O argumento 'plot_kws' permite personalizar os plots fora da diagonal (scatter plots)

    # Exemplo de customização: usar KDE nos diagonais e scatter plots com transparência
    g = sns.pairplot(
        combined_df,
        hue='Data_Source',        # Coluna para diferenciar os pontos por cor
        vars=cols_to_plot,        # As colunas a serem plotadas
        diag_kind='kde',          # Tipo de plot na diagonal (Kernel Density Estimate)
        plot_kws={'alpha': 0.6, 's': 10}, # alpha: transparência, s: tamanho do ponto
        height=2.5                # Altura de cada subplot
    )

    # Ajustar o título
    g.fig.suptitle('Comparação Pair Plot: Dados Reais vs. Sintéticos', y=1.02, fontsize=16) # Adiciona título acima

    # Melhorar a legenda (opcional, pairplot cria uma por padrão)
    # g.add_legend()

    plt.show()

    # --- Opcional: Pair plot com histograma na diagonal ---
    print("\nGerando Pair Plot com Histograma na diagonal...")
    g_hist = sns.pairplot(
        combined_df,
        hue='Data_Source',
        vars=cols_to_plot,
        diag_kind='hist',        # Tipo de plot na diagonal (Histograma)
        plot_kws={'alpha': 0.6, 's': 10},
        height=2.5
    )
    g_hist.fig.suptitle('Comparação Pair Plot (Hist): Dados Reais vs. Sintéticos', y=1.02, fontsize=16)
    plt.show()

In [ ]:
# seaborn.pairplot pode ser usado para visualizar pares de colunas selecionadas
sns.pairplot(processed_df[['RIDE_HEIGHT_FA_DIFF', 'RIDE_HEIGHT_FR', 'RIDE_HEIGHT_FL_INITIAL']])
plt.suptitle('Pair Plot for Selected Toe Columns (Real Data)', y=1.02) # Adiciona título acima
plt.show()

In [ ]:
# seaborn.pairplot pode ser usado para visualizar pares de colunas selecionadas
sns.pairplot(synthetic_data[['RIDE_HEIGHT_FA_DIFF', 'RIDE_HEIGHT_FR', 'RIDE_HEIGHT_FL_INITIAL']])
plt.suptitle('Pair Plot for Selected Toe Columns (Synthetic Data)', y=1.02) # Adiciona título acima
plt.show()

In [ ]:


# --- Passo 1: Selecionar as colunas de interesse ---
# Certifique-se de que as colunas existem em ambos os DataFrames
cols_to_plot = ['RIDE_HEIGHT_FA_DIFF', 'RIDE_HEIGHT_FR', 'RIDE_HEIGHT_FL_INITIAL']

# Verificar se as colunas existem em ambos os DataFrames antes de prosseguir
if not all(col in processed_df.columns for col in cols_to_plot):
    print(f"Erro: Algumas colunas não encontradas no DataFrame real: {[col for col in cols_to_plot if col not in processed_df.columns]}")
elif not all(col in synthetic_data.columns for col in cols_to_plot):
     print(f"Erro: Algumas colunas não encontradas no DataFrame sintético: {[col for col in cols_to_plot if col not in synthetic_data.columns]}")
else:
    # --- Passo 2: Preparar os DataFrames para a comparação ---

    # Selecionar apenas as colunas relevantes de cada DataFrame
    real_subset = processed_df[cols_to_plot].copy()
    synthetic_subset = synthetic_data[cols_to_plot].copy()

    # Adicionar uma coluna para identificar a origem dos dados
    real_subset['Data_Source'] = 'Real'
    synthetic_subset['Data_Source'] = 'Synthetic'

    # Combinar os dois DataFrames
    combined_df = pd.concat([real_subset, synthetic_subset], ignore_index=True)

    print("DataFrames combinados para pair plot com sucesso.")

    # --- Passo 3: Gerar o Pair Plot Comparativo ---

    # Use seaborn.pairplot no DataFrame combinado
    # O argumento 'hue' plota pontos com cores diferentes baseadas na coluna 'Data_Source'
    # O argumento 'diag_kws' permite personalizar os plots diagonais (geralmente histogramas ou kde)
    # O argumento 'plot_kws' permite personalizar os plots fora da diagonal (scatter plots)

    # Exemplo de customização: usar KDE nos diagonais e scatter plots com transparência
    g = sns.pairplot(
        combined_df,
        hue='Data_Source',        # Coluna para diferenciar os pontos por cor
        vars=cols_to_plot,        # As colunas a serem plotadas
        diag_kind='kde',          # Tipo de plot na diagonal (Kernel Density Estimate)
        plot_kws={'alpha': 0.6, 's': 10}, # alpha: transparência, s: tamanho do ponto
        height=2.5                # Altura de cada subplot
    )

    # Ajustar o título
    g.fig.suptitle('Comparação Pair Plot: Dados Reais vs. Sintéticos', y=1.02, fontsize=16) # Adiciona título acima

    # Melhorar a legenda (opcional, pairplot cria uma por padrão)
    # g.add_legend()

    plt.show()

    # --- Opcional: Pair plot com histograma na diagonal ---
    print("\nGerando Pair Plot com Histograma na diagonal...")
    g_hist = sns.pairplot(
        combined_df,
        hue='Data_Source',
        vars=cols_to_plot,
        diag_kind='hist',        # Tipo de plot na diagonal (Histograma)
        plot_kws={'alpha': 0.6, 's': 10},
        height=2.5
    )
    g_hist.fig.suptitle('Comparação Pair Plot (Hist): Dados Reais vs. Sintéticos', y=1.02, fontsize=16)
    plt.show()

In [ ]:
# --- Contando as colunas por tipo ---

# 1. Contar colunas numéricas:
# Usamos np.number para pegar todos os tipos numéricos (int, float, etc.)
num_cols = synthetic_data.select_dtypes(include=np.number).columns
count_numeric = len(num_cols)

# 2. Contar colunas categóricas:
# Incluímos 'object' (geralmente strings) e 'category' (tipo categórico explícito do pandas)
cat_cols = synthetic_data.select_dtypes(include=['object', 'category']).columns
count_categorical = len(cat_cols)

# 3. Contar colunas timestamp:
# Incluímos 'datetime64' para pegar os tipos de data e hora
ts_cols = synthetic_data.select_dtypes(include='datetime64').columns
count_timestamp = len(ts_cols)

# --- Exibindo os resultados ---
print(f"Total de colunas numéricas: {count_numeric}")
# Opcional: ver quais são: print(f"Colunas numéricas: {list(num_cols)}")

print(f"Total de colunas categóricas: {count_categorical}")
# Opcional: ver quais são: print(f"Colunas categóricas: {list(cat_cols)}")

print(f"Total de colunas timestamp: {count_timestamp}")
# Opcional: ver quais são: print(f"Colunas timestamp: {list(ts_cols)}")

# Opcional: Contar outras colunas (que não se encaixam nas categorias acima)
count_other = len(synthetic_data.columns) - (count_numeric + count_categorical + count_timestamp)
print(f"Total de outras colunas: {count_other}")

In [ ]:
df.columns

In [ ]:
!pip install prince

In [ ]:
categorical_cols_for_mca = [
    'Vehicle_Type',
    'TRIM_SERIES',
    'WHEELBASE',
    'Drive_',      # Se essas foram transformadas
    'Vehicle_',
    'Engine_',
    'Cab_',
    'CabStyle',
    'Engine',
    'RearWheels',
    'SteeringGear',
    'Suspension',
    'Tire',
    'Transmission',
    'Trim_Series_broadcast',
    'Wheel',
    'wheelbase_broadcast',
    'LaneDeparture',
    'MY',
    'Warranty', # 'Warranty' também é categórica
    'TIS'
]

# Opcional: Filtre para garantir que essas colunas existam no processed_df
categorical_cols_for_mca = [col for col in categorical_cols_for_mca if col in processed_df.columns]

print(f"Colunas categóricas selecionadas para MCA: {categorical_cols_for_mca}")

# Crie um sub-DataFrame apenas com as colunas categóricas
df_categorical = processed_df[categorical_cols_for_mca].copy()

In [ ]:
# Verifique se há NaNs nas colunas categóricas. MCA geralmente não lida com NaNs.
# Se houver, você precisará tratá-los (preencher com moda, por exemplo).
print("\nVerificando NaNs em colunas categóricas:")
print(df_categorical.isnull().sum())

# Se houver NaNs, preencha-os (ex: com a moda)
for col in df_categorical.columns:
    if df_categorical[col].isnull().any():
        mode_val = df_categorical[col].mode()
        if not mode_val.empty:
            df_categorical[col].fillna(mode_val[0], inplace=True)
            print(f"Preenchendo NaNs em '{col}' com a moda: '{mode_val[0]}'")
        else:
            # Trate colunas onde a moda não pôde ser calculada (todos NaNs ou valores únicos)
            # Pode ser necessário dropar a coluna ou preencher com um valor placeholder
             df_categorical[col].fillna('NaN_Filled', inplace=True)
             print(f"Preenchendo NaNs em '{col}' com 'NaN_Filled' (moda vazia).")


In [ ]:
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import prince
import plotly.io as pio
pio.renderers.default = 'browser'
import plotly.graph_objects as go
from itertools import combinations


In [ ]:
# Rename the columns to not interfere the algorithm to plot data
df_categorical.rename(columns=lambda x: x.replace('_', '.'), inplace=True)

In [ ]:
#create lists to append columns that's import or not
p_value_h1 = []
p_value_h0 = []
chi2s = dict()

# Combine and apply Chi-Square Test into all columns
for item in list(combinations(df_categorical.columns, 2)):
    #print(item, '\n')

    table = pd.crosstab(df_categorical[item[0]], df_categorical[item[1]])
    #print(table)

    chi2, pvalue, gl, freq_esp = chi2_contingency(table)
    #print(f"Qui² Statistic: {round(chi2, 2)}")
    #print(f"p-value: {round(pvalue, 4)}", "\n")
    if 'TIS' in item :
        if pvalue < 0.05:
            p_value_h1= [item[0] + item[1]]
            key_col = ', '.join(p_value_h1)
            chi2s[key_col] = chi2

    else:
        p_value_h0.append(item[0])
        p_value_h0.append(item[1])
        p_value_h0.append('\n')

In [ ]:
df_categorical.columns

In [ ]:
#print all possible combination that accept H1 and Reject H0
print(f"List of Combinations accepting H1: {chi2s}")

In [ ]:
chi2s

In [ ]:
for k in chi2s:
    value = chi2s[k]
    if value > 500:
        print(k)

In [ ]:
df_obj = df_categorical[['Vehicle.Type','TRIM.SERIES','Engine','SteeringGear','Tire','Wheel','LaneDeparture', 'MY', 'TIS', 'Warranty']]

In [ ]:
#Apply the unsupervised machine learning model
mca = prince.MCA(n_components=2).fit(df_obj)

In [ ]:
# Dimension Quantity
quant_dim = mca.J_ - mca.K_

In [ ]:
print(mca.total_inertia_/quant_dim)

In [ ]:
print(f"Total of Categories: {mca.J_}")
print(f"Total of Variables: {mca.K_}")
print(f"Total of Dimensions: {quant_dim}")

In [ ]:
# the eigenvalues
table_eigenvalues = mca.eigenvalues_summary
table_eigenvalues

In [ ]:
#Total main inertia
#Sum of all eigenvalues (all existing dimensions)

print(mca.total_inertia_)

In [ ]:
# Getting the main coordinates of the variable categories
coord_burt = mca.column_coordinates(df_obj)

In [ ]:
# the coordinates of the observations come from the standard coordinates
coord_obs = mca.row_coordinates(df_obj)

In [ ]:
# Obtaining the standard coordinates of variable categories
coord_padrao = mca.column_coordinates(df_obj)/np.sqrt(mca.eigenvalues_)

In [ ]:
# Plotting the perceptual map (standard coordinates)
# Note: for the above function to be executed properly, do not leave an underline in the original name of the variable in the dataset!
chart = coord_padrao.reset_index()

nome_categ=[]
for col in df_obj:
    nome_categ.append(df_obj[col].sort_values(ascending=True).unique())
    categorias = pd.DataFrame(nome_categ).stack().reset_index()

var_chart = pd.Series(chart['index'].str.split('_', expand=True).iloc[:,0])

chart_df_mca = pd.DataFrame({'category': chart['index'],
                             'obs_x': chart[0],
                             'obs_y': chart[1],
                             'variable': var_chart,
                             'category_id': categorias[0]})

In [ ]:
chart

In [ ]:
chart_df_mca

In [ ]:
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'] + 0.03, point['y'] - 0.02, point['val'], fontsize=5)

label_point(x = chart_df_mca['obs_x'],
            y = chart_df_mca['obs_y'],
            val = chart_df_mca['category_id'],
            ax = plt.gca())


sns.scatterplot(data=chart_df_mca, x='obs_x', y='obs_y', hue='variable', s=20)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.axhline(y=0, color='lightgrey', ls='--', linewidth=0.8)
plt.axvline(x=0, color='lightgrey', ls='--', linewidth=0.8)
plt.tick_params(size=2, labelsize=6)
plt.legend(bbox_to_anchor=(1.25,-0.2), fancybox=True, shadow=True, ncols=10, fontsize='5')
plt.title("Perceptual Map - MCA", fontsize=12)
plt.xlabel(f"Dim. 1: {table_eigenvalues.iloc[0,1]} of Inertia", fontsize=8)
plt.ylabel(f"Dim. 2: {table_eigenvalues.iloc[1,1]} of Inertia", fontsize=8)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import prince
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px # Plotly pode ser melhor para biplots interativos
import plotly.graph_objects as go
import plotly.io as pio

# Configurar o renderer do Plotly para exibir no ambiente Colab/Jupyter
pio.renderers.default = 'colab'

# --- Passo 1: Identificar e Preparar Colunas Categóricas (igual ao código anterior) ---

# Liste manualmente as colunas que você sabe que são categóricas e transformadas
# ou identifique-as programaticamente.
#categorical_cols_for_mca = ['Vehicle.Type','TRIM.SERIES','Engine','SteeringGear','Tire','Wheel','LaneDeparture', 'MY', 'TIS', 'Warranty']
categorical_cols_for_mca = [
    'Vehicle_Type',
    'TRIM_SERIES',
    'WHEELBASE',
    'Drive_',      # Se essas foram transformadas
    'Vehicle_',
    'Engine_',
    'Cab_',
    'CabStyle',
    'Engine',
    'RearWheels',
    'SteeringGear',
    'Suspension',
    'Tire',
    'Transmission',
    'Trim_Series_broadcast',
    'Wheel',
    'wheelbase_broadcast',
    'LaneDeparture',
    'MY',
    'Warranty', # 'Warranty' também é categórica
    'TIS'
]

# Opcional: Filtre para garantir que essas colunas existam em ambos os DataFrames
# e no processed_df (que é o que usaremos como base)
cols_present_in_real = [col for col in categorical_cols_for_mca if col in processed_df.columns]
cols_present_in_synthetic = [col for col in cols_present_in_real if col in synthetic_data.columns]



if len(cols_present_in_synthetic) < len(categorical_cols_for_mca):
    missing_cols = [col for col in categorical_cols_for_mca if col not in cols_present_in_synthetic]
    print(f"Aviso: As seguintes colunas categóricas selecionadas não estão presentes em ambos os DataFrames (Real e Sintético): {missing_cols}. Elas serão excluídas da análise MCA.")

categorical_cols_for_mca = cols_present_in_synthetic

if not categorical_cols_for_mca:
    print("Erro: Nenhuma coluna categórica comum encontrada nos DataFrames Real e Sintético para realizar a MCA.")
else:
    print(f"Colunas categóricas comuns selecionadas para MCA: {categorical_cols_for_mca}")

    # Crie sub-DataFrames apenas com as colunas categóricas comuns
    df_categorical_real = processed_df[categorical_cols_for_mca].copy()
    df_categorical_synthetic = synthetic_data[categorical_cols_for_mca].copy()

    # Rename the columns to not interfere the algorithm to plot data
    df_categorical_real.rename(columns=lambda x: x.replace('_', '.'), inplace=True)
    df_categorical_synthetic.rename(columns=lambda x: x.replace('_', '.'), inplace=True)

    # Tratar NaNs (se necessário, igual ao código anterior)
    print("\nTratando NaNs em colunas categóricas (Real)...")
    for col in df_categorical_real.columns:
        if df_categorical_real[col].isnull().any():
            mode_val = df_categorical_real[col].mode()
            if not mode_val.empty:
                df_categorical_real[col].fillna(mode_val[0], inplace=True)
            else:
                df_categorical_real[col].fillna('NaN_Filled_Real', inplace=True)

    print("Tratando NaNs em colunas categóricas (Sintético)...")
    for col in df_categorical_synthetic.columns:
        if df_categorical_synthetic[col].isnull().any():
            mode_val = df_categorical_synthetic[col].mode()
            if not mode_val.empty:
                df_categorical_synthetic[col].fillna(mode_val[0], inplace=True)
            else:
                df_categorical_synthetic[col].fillna('NaN_Filled_Synthetic', inplace=True)


    # --- Passo 2: Treinar Modelos MCA Separados ---

    # Inicializar e Ajustar o Modelo MCA para Dados Reais
    mca_real = prince.MCA(n_components=2, n_iter=10, random_state=42)
    print("\nAjustando o modelo MCA para Dados Reais...")
    mca_real = mca_real.fit(df_categorical_real)
    print("Ajuste MCA (Real) completo.")

    # Inicializar e Ajustar o Modelo MCA para Dados Sintéticos
    mca_synthetic = prince.MCA(n_components=2, n_iter=10, random_state=42)
    print("Ajustando o modelo MCA para Dados Sintéticos...")
    mca_synthetic = mca_synthetic.fit(df_categorical_synthetic)
    print("Ajuste MCA (Sintético) completo.")

    # --- Passo 3: Obter Coordenadas das Categorias e Inércia ---

    # Coordenadas das categorias para Dados Reais
    category_coords_real = mca_real.column_coordinates(df_categorical_real)
    category_coords_real['Data_Source'] = 'Real'
    category_coords_real.columns = ['Dim 1', 'Dim 2', 'Data_Source']

    # Coordenadas das categorias para Dados Sintéticos
    category_coords_synthetic = mca_synthetic.column_coordinates(df_categorical_synthetic)
    category_coords_synthetic['Data_Source'] = 'Synthetic'
    category_coords_synthetic.columns = ['Dim 1', 'Dim 2', 'Data_Source']

    # Combinar as coordenadas das categorias para plotagem comparativa
    combined_category_coords = pd.concat([category_coords_real, category_coords_synthetic])

    # Obter a inércia explicada
    # Calcular a inércia explicada usando a fórmula: eigenvalues_ / total_inertia_
    total_inertia_real = mca_real.total_inertia_
    explained_inertia_real = mca_real.eigenvalues_ / total_inertia_real

    total_inertia_synthetic = mca_synthetic.total_inertia_
    explained_inertia_synthetic = mca_synthetic.eigenvalues_ / total_inertia_synthetic



    print("\nInércia explicada por dimensão (Real vs. Sintético):")
    for i in range(min(len(explained_inertia_real), len(explained_inertia_synthetic))):
        print(f"  Dimensão {i+1}: Real={explained_inertia_real[i]:.4f}, Sintético={explained_inertia_synthetic[i]:.4f}")


    # --- Passo 4: Visualizar as Coordenadas das Categorias Lado a Lado ou Sobrepostas ---

    print("\nGerando Scatter Plot Comparativo para Coordenadas das Categorias...")

    # Opção 1: Scatter Plot Comparativo (Real e Sintético no mesmo gráfico, diferenciados por cor)
    # Use Plotly para um gráfico interativo (útil para ver os nomes das categorias ao passar o mouse)
    fig = px.scatter(
        combined_category_coords.reset_index(), # Resetar índice para que o nome da categoria vire coluna
        x='Dim 1',
        y='Dim 2',
        color='Data_Source', # Diferenciar por cor
        text='index',       # Usar o nome da categoria como texto para rótulos
        title='MCA - Comparação de Coordenadas das Categorias (Real vs. Sintético)',
        labels={'index': 'Categoria'}, # Rótulo para o tooltip
        hover_name='index', # Mostrar o nome da categoria no tooltip
        size_max=10 # Tamanho máximo dos pontos
    )

    # Melhorar o layout (opcional)
    fig.update_traces(textposition='top center') # Posição dos rótulos
    fig.update_layout(
        xaxis_title=f'Dimensão 1 (Real: {explained_inertia_real[0]*100:.2f}%, Sintético: {explained_inertia_synthetic[0]*100:.2f}%)',
        yaxis_title=f'Dimensão 2 (Real: {explained_inertia_real[1]*100:.2f}%, Sintético: {explained_inertia_synthetic[1]*100:.2f}%)',
        hovermode='closest' # Melhorar o comportamento do tooltip
    )
    fig.show()


    # Opção 2: Scatter Plots Lado a Lado (usando Matplotlib)
    # Se preferir Matplotlib, você pode plotar os dois conjuntos de coordenadas separadamente
    # em subplots, similar ao heatmap de correlação.

    fig_mpl, axes_mpl = plt.subplots(1, 2, figsize=(20, 10)) # 1 linha, 2 colunas

    # Plotar Coordenadas Reais
    sns.scatterplot(data=category_coords_real, x='Dim 1', y='Dim 2', ax=axes_mpl[0])
    for i, (index, row) in enumerate(category_coords_real.iterrows()):
         axes_mpl[0].text(row['Dim 1'] + 0.005, row['Dim 2'] + 0.005, index, fontsize=8)
    axes_mpl[0].set_title('MCA - Coordenadas das Categorias (Real)')
    axes_mpl[0].set_xlabel(f'Dimensão 1 ({explained_inertia_real[0]*100:.2f}% Inércia)')
    axes_mpl[0].set_ylabel(f'Dimensão 2 ({explained_inertia_real[1]*100:.2f}% Inércia)')
    axes_mpl[0].grid(True, linestyle='--', alpha=0.6)
    axes_mpl[0].axhline(0, color='gray', linestyle='-', linewidth=0.8)
    axes_mpl[0].axvline(0, color='gray', linestyle='-', linewidth=0.8)


    # Plotar Coordenadas Sintéticas
    sns.scatterplot(data=category_coords_synthetic, x='Dim 1', y='Dim 2', ax=axes_mpl[1])
    for i, (index, row) in enumerate(category_coords_synthetic.iterrows()):
         axes_mpl[1].text(row['Dim 1'] + 0.005, row['Dim 2'] + 0.005, index, fontsize=8)
    axes_mpl[1].set_title('MCA - Coordenadas das Categorias (Sintético)')
    axes_mpl[1].set_xlabel(f'Dimensão 1 ({explained_inertia_synthetic[0]*100:.2f}% Inércia)')
    axes_mpl[1].set_ylabel(f'Dimensão 2 ({explained_inertia_synthetic[1]*100:.2f}% Inércia)')
    axes_mpl[1].grid(True, linestyle='--', alpha=0.6)
    axes_mpl[1].axhline(0, color='gray', linestyle='-', linewidth=0.8)
    axes_mpl[1].axvline(0, color='gray', linestyle='-', linewidth=0.8)

    plt.tight_layout()
    plt.show()

    print("\n--- Análise Comparativa ---")
    print("Observe a proximidade dos pontos de categorias correspondentes nos gráficos (especialmente no Plotly interativo ou nos gráficos lado a lado do Matplotlib).")
    print("Se uma categoria (ex: 'Engine_V8') está próxima de outra (ex: 'Transmission_0') nos dados reais e as categorias correspondentes ('Engine_V8', 'Transmission_0') estão próximas nos dados sintéticos, isso indica que a associação foi bem preservada.")
    print("Compare também a inércia explicada pelas primeiras dimensões. Valores semelhantes sugerem que a estrutura geral das associações foi replicada.")

In [ ]:
import pandas as pd
import numpy as np
import prince
# Importar bibliotecas de visualização, se necessário para plotar as diferenças
import matplotlib.pyplot as plt
import seaborn as sns

# Assumindo que você já executou o código anterior e tem:
# processed_df (seu DataFrame real limpo e com colunas renomeadas '_ ' para '.')
# synthetic_data (seu DataFrame sintético)
# mca_real (modelo MCA ajustado nos dados reais)
# mca_synthetic (modelo MCA ajustado nos dados sintéticos)

# --- Passo 1: Obter as Coordenadas das Categorias de Ambos os Modelos ---

# Certifique-se de que as colunas categóricas usadas para treinar o MCA são as mesmas
# e que foram renomeadas consistentemente (underline para ponto).
# Vamos reutilizar a lista de colunas usadas no bloco anterior de comparação de MCA
categorical_cols_for_mca_renamed = [
    'Vehicle.Type', 'TRIM.SERIES', 'WHEELBASE', 'Drive.', 'Vehicle.',
    'Engine.', 'Cab.', 'CabStyle', 'Engine', 'RearWheels', 'SteeringGear',
    'Suspension', 'Tire', 'Transmission', 'Trim.Series.broadcast',
    'Wheel', 'wheelbase.broadcast', 'LaneDeparture', 'MY', 'TIS', 'Warranty'
]
# Filtrar para garantir que essas colunas renomeadas existem nos DataFrames MCA
# que foram usados para gerar as coordenadas (assumimos que são df_categorical_real/synthetic)
# e que os modelos mca_real/synthetic foram treinados com essas colunas.
# Nota: O renomeamento para '.' é crucial para que as coordenadas tenham os mesmos nomes.

try:
    # Obter as coordenadas das categorias (padrão ou principais)
    # As 'standard coordinates' (divididas pela raiz do eigenvalue) são boas para visualização
    # As 'principal coordinates' (coordenadas de Burt) também podem ser usadas
    # Vamos usar as coordenadas principais (de Burt) aqui para comparação direta de posição.
    # Certifique-se de que as colunas usadas aqui correspondem às que foram usadas para treinar o MCA.
    # Assume-se que mca_real e mca_synthetic foram treinados em df_categorical_real e df_categorical_synthetic
    # que já tiveram suas colunas renomeadas para '.'

    category_coords_real = mca_real.column_coordinates(df_categorical_real)
    category_coords_synthetic = mca_synthetic.column_coordinates(df_categorical_synthetic)

    # Renomear as colunas de dimensão para serem consistentes (Dim 1, Dim 2)
    category_coords_real.columns = [f'Dim {i+1}' for i in range(category_coords_real.shape[1])]
    category_coords_synthetic.columns = [f'Dim {i+1}' for i in range(category_coords_synthetic.shape[1])]

    print(f"Coordenadas Reais obtidas: Shape {category_coords_real.shape}")
    print(f"Coordenadas Sintéticas obtidas: Shape {category_coords_synthetic.shape}")

except NameError:
    print("Erro: Certifique-se de que 'processed_df', 'synthetic_data', 'mca_real', 'mca_synthetic', 'df_categorical_real', e 'df_categorical_synthetic' foram definidos e que os modelos MCA foram ajustados antes de executar este bloco.")
    # Se ocorrer um erro, pare a execução deste bloco
    raise

# --- Passo 2: Identificar Categorias Comuns em Ambas as Coordenadas ---

# Os índices dos DataFrames de coordenadas são os nomes das categorias.
common_categories = list(category_coords_real.index.intersection(category_coords_synthetic.index))

if not common_categories:
    print("\nErro: Nenhuma categoria comum encontrada nos resultados de coordenadas MCA. Verifique se os modelos MCA foram treinados em DataFrames com nomes de colunas e categorias consistentes.")
else:
    print(f"\nComparando coordenadas para {len(common_categories)} categorias comuns.")

    # Filtrar as coordenadas para incluir apenas as categorias comuns
    coords_real_common = category_coords_real.loc[common_categories]
    coords_synthetic_common = category_coords_synthetic.loc[common_categories]

    # Garantir que a ordem das categorias é a mesma para comparação ponto a ponto
    coords_real_common = coords_real_common.sort_index()
    coords_synthetic_common = coords_synthetic_common.sort_index()


    # --- Passo 3: Calcular a Distância entre as Coordenadas Correspondentes ---

    # Calcular a distância Euclidiana para as primeiras n_components (Dim 1 e Dim 2)
    # Você pode estender isso para mais dimensões se n_components > 2
    n_dims_to_compare = min(mca_real.n_components, mca_synthetic.n_components)
    print(f"Calculando distância Euclidiana usando as primeiras {n_dims_to_compare} dimensões.")

    # Calcular a diferença nas coordenadas
    coord_difference = coords_real_common.iloc[:, :n_dims_to_compare] - coords_synthetic_common.iloc[:, :n_dims_to_compare]

    # Calcular a distância Euclidiana (raiz quadrada da soma dos quadrados das diferenças)
    # axis=1 soma ao longo das colunas para cada linha (categoria)
    distances = np.sqrt((coord_difference**2).sum(axis=1))

    # Criar um DataFrame para visualizar as distâncias
    distance_df = pd.DataFrame({
        'Categoria': distances.index,
        'Distancia_Euclidiana': distances.values
    })

    # Ordenar por distância para ver quais categorias têm as maiores/menores diferenças
    distance_df = distance_df.sort_values(by='Distancia_Euclidiana', ascending=False).reset_index(drop=True)

    print("\n--- Maiores Diferenças nas Coordenadas das Categorias (Distância Euclidiana) ---")
    # Exibir as categorias com maiores distâncias (estrutura menos bem replicada)
    print(distance_df.head(10)) # Exibir as 10 maiores diferenças

    print("\n--- Menores Diferenças nas Coordenadas das Categorias (Distância Euclidiana) ---")
    # Exibir as categorias com menores distâncias (estrutura mais bem replicada)
    print(distance_df.tail(10)) # Exibir as 10 menores diferenças (excluindo possivelmente 0 se forem idênticas)


    # --- Passo 4: Análise Visual (Opcional) ---

    # Você pode querer visualizar a distribuição dessas distâncias
    plt.figure(figsize=(8, 5))
    sns.histplot(distance_df['Distancia_Euclidiana'], kde=True)
    plt.title('Distribuição das Distâncias nas Coordenadas das Categorias (Real vs. Sintético)')
    plt.xlabel('Distância Euclidiana nas Coordenadas MCA (Dim 1 e Dim 2)')
    plt.ylabel('Frequência')
    plt.show()

    # Ou um boxplot para resumir
    plt.figure(figsize=(5, 6))
    sns.boxplot(y=distance_df['Distancia_Euclidiana'])
    plt.title('Boxplot das Distâncias nas Coordenadas das Categorias')
    plt.ylabel('Distância Euclidiana')
    plt.show()


    print("\n--- Análise das Distâncias ---")
    print(f"Distância Euclidiana Média: {distance_df['Distancia_Euclidiana'].mean():.4f}")
    print(f"Distância Euclidiana Mediana: {distance_df['Distancia_Euclidiana'].median():.4f}")
    print(f"Distância Euclidiana Máxima: {distance_df['Distancia_Euclidiana'].max():.4f}")

    print("\nInterpretação:")
    print("- Distâncias pequenas indicam que a posição (e, portanto, as principais associações) daquela categoria foi bem replicada pelo modelo sintético.")
    print("- Distâncias grandes indicam que a posição (e associações) daquela categoria é significativamente diferente no dataset sintético.")
    print("- A distribuição geral das distâncias e a média/mediana dão uma ideia do quão bem a estrutura de associação CATEGRICAL geral foi replicada.")
    print("- Categorias com as maiores distâncias são aquelas onde o modelo sintético falhou mais em replicar sua estrutura de associação.")

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# Configurar o renderer do Plotly
pio.renderers.default = 'colab'

# --- Assumindo que você já executou o código anterior e tem: ---
# category_coords_real (DataFrame com coordenadas das categorias Reais, colunas 'Dim 1', 'Dim 2', ...)
# category_coords_synthetic (DataFrame com coordenadas das categorias Sintéticas, colunas 'Dim 1', 'Dim 2', ...)
# common_categories (lista das categorias presentes em ambos os DataFrames de coordenadas)
# distance_df (DataFrame com 'Categoria' e 'Distancia_Euclidiana')

# --- Passo 1: Preparar os Dados para o Plot de Vetores ---

if 'common_categories' not in locals() or not common_categories:
     print("Erro: A lista 'common_categories' não foi encontrada ou está vazia. Execute os blocos anteriores para obter as coordenadas e identificar as categorias comuns.")
else:
    # Filtrar as coordenadas para incluir apenas as categorias comuns e garantir a ordem
    coords_real_common = category_coords_real.loc[common_categories, ['Dim 1', 'Dim 2']].sort_index()
    coords_synthetic_common = category_coords_synthetic.loc[common_categories, ['Dim 1', 'Dim 2']].sort_index()
    distance_df_common = distance_df.set_index('Categoria').loc[common_categories].sort_index() # Garantir que distance_df tbm esteja na mesma ordem

    # Criar um DataFrame que combine as coordenadas Real e Sintética e a distância
    plot_data = pd.DataFrame({
        'Categoria': common_categories,
        'x_real': coords_real_common['Dim 1'].values,
        'y_real': coords_real_common['Dim 2'].values,
        'x_synthetic': coords_synthetic_common['Dim 1'].values,
        'y_synthetic': coords_synthetic_common['Dim 2'].values,
        'distance': distance_df_common['Distancia_Euclidiana'].values
    })

    # Adicionar colunas para o final dos vetores (x_synthetic - x_real, y_synthetic - y_real)
    # Ou podemos simplesmente plotar uma linha do real para o sintético
    # Vamos plotar linhas para conectar o ponto real ao ponto sintético

    print(f"\nPreparando dados para plotagem de diferenças para {len(plot_data)} categorias comuns.")

    # --- Passo 2: Criar o Gráfico Interativo com Plotly ---

    fig = go.Figure()

    # Adicionar os pontos Reais
    fig.add_trace(go.Scattergl( # Usando Scattergl para melhor performance com muitos pontos
        x=plot_data['x_real'],
        y=plot_data['y_real'],
        mode='markers',
        marker=dict(
            size=5,
            opacity=0.6,
            color='blue'
        ),
        name='Real',
        hoverinfo='text',
        text=[f'Real: {cat}<br>Dim 1: {x:.3f}<br>Dim 2: {y:.3f}'
              for cat, x, y in zip(plot_data['Categoria'], plot_data['x_real'], plot_data['y_real'])]
    ))

    # Adicionar os pontos Sintéticos
    fig.add_trace(go.Scattergl(
        x=plot_data['x_synthetic'],
        y=plot_data['y_synthetic'],
        mode='markers',
        marker=dict(
            size=5,
            opacity=0.6,
            color='red'
        ),
        name='Synthetic',
         hoverinfo='text',
        text=[f'Synthetic: {cat}<br>Dim 1: {x:.3f}<br>Dim 2: {y:.3f}'
              for cat, x, y in zip(plot_data['Categoria'], plot_data['x_synthetic'], plot_data['y_synthetic'])]
    ))

    # Adicionar as linhas (vetores conceituais) conectando Real ao Sintético
    # Iterar sobre as linhas do DataFrame plot_data
    for index, row in plot_data.iterrows():
        fig.add_trace(go.Scatter(
            x=[row['x_real'], row['x_synthetic']],
            y=[row['y_real'], row['y_synthetic']],
            mode='lines',
            line=dict(
                width=0.8,
                color=f'rgba(0, 0, 0, {0.2 + 0.8 * (row["distance"] / distance_df["Distancia_Euclidiana"].max())})' # Opacidade baseada na distância
                 # Você pode mapear a cor para a distância tbm
            ),
            showlegend=False, # Não mostrar legenda para cada linha
            hoverinfo='text',
            text=f'Categoria: {row["Categoria"]}<br>Distância: {row["distance"]:.4f}'
        ))

    # --- Configurar Layout do Gráfico ---

    fig.update_layout(
        title='MCA - Comparação da distância das Coordenadas das Categorias (Real vs. Sintético)',
        xaxis_title='Dimensão 1', # Poderia adicionar a inércia explicada aqui, mas o título ficaria longo
        yaxis_title='Dimensão 2',
        hovermode='closest', # Melhorar o comportamento do tooltip
        showlegend=True,
        width=800, # Ajuste o tamanho conforme necessário
        height=600
    )

    # Adicionar linhas de referência nos eixos 0,0
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_vline(x=0, line_dash="dash", line_color="gray", opacity=0.5)


    # Exibir o gráfico
    print("\nExibindo o gráfico interativo de diferenças...")
    fig.show()

In [ ]:


# --- Assumindo que você já executou o código anterior que calculou as distâncias ---
# E que o DataFrame 'distance_df' foi criado e contém as colunas 'Categoria' e 'Distancia_Euclidiana'.

# Verificar se o DataFrame 'distance_df' existe
if 'distance_df' in locals():
    print("--- Tabela de Diferenças Absolutas nas Coordenadas das Categorias (Sintético vs Real) ---")

    # O DataFrame 'distance_df' já está ordenado em ordem decrescente pela distância
    # E já tem o índice resetado, então está pronto para exibição.
    # Renomear a coluna de distância para clareza no output
    table_of_differences = distance_df.rename(columns={'Distancia_Euclidiana': 'Distancia_Euclidiana_Real_vs_Sintetico'})

    # Exibir a tabela completa ou as primeiras/últimas linhas se for muito grande
    # Para exibir a tabela completa:
    # print(table_of_differences.to_string())

    # Para exibir as primeiras 15 e últimas 15 linhas (se houver muitas categorias):
    if len(table_of_differences) > 30:
        print("Exibindo as 15 maiores e 15 menores diferenças:")
        print(table_of_differences.head(15).to_string(index=False))
        print("\n...\n")
        print(table_of_differences.tail(15).to_string(index=False))
    else:
        # Se a tabela for pequena, exibe tudo
        print(table_of_differences.to_string(index=False))

    print("\n--- Fim da Tabela ---")

else:
    print("Erro: O DataFrame 'distance_df' não foi encontrado. Certifique-se de executar o código anterior que calcula as distâncias das coordenadas.")

In [ ]:
table_of_differences

In [ ]:

# --- Assumindo que 'table_of_differences' foi criada no bloco anterior ---
# E contém as colunas 'Categoria' e 'Distancia_Euclidiana_Real_vs_Sintetico'.

if 'table_of_differences' in locals():
    print("--- Análise de Variáveis com Maiores Diferenças nas Coordenadas ---")

    # Criar uma cópia para não modificar o DataFrame original se necessário
    diff_analysis_df = table_of_differences.copy()

    # Dividir a coluna 'Categoria' no delimitador '__' e pegar a primeira parte
    # Usamos str.split('__', expand=True) para criar colunas separadas
    # e pegamos a primeira coluna (iloc[:, 0])
    # Lidamos com o caso onde '__' não existe (o split retornaria apenas uma parte)
    diff_analysis_df['Nome_Variavel'] = diff_analysis_df['Categoria'].str.split('__', expand=True).iloc[:, 0]

    # Se após o split a parte for igual ao nome original da categoria,
    # pode ser que não houvesse '__'. Precisamos pensar em como tratar isso.
    # No seu código original de transformação, o formato era 'columnname_index'.
    # No MCA com colunas renomeadas para '.', o formato seria 'columnname.index'.
    # Vamos assumir que você quer separar o nome original da variável do índice numérico gerado.
    # O split deve ser feito no último '.' se o formato for 'NomeVariavel.indice'.

    # Corrigindo o split baseado no formato 'NomeVariavel.indice' após renomeamento para '.'
    # Vamos dividir no último '.' e pegar a parte ANTES do último '.'
    # Ex: 'Vehicle.Type.0' -> 'Vehicle.Type'
    # Ex: 'MY.2022' -> 'MY'
    # Ex: 'TIS.0' -> 'TIS'
    # Vamos usar rsplit para dividir apenas no último ponto.
    diff_analysis_df['Nome_Variavel_Original'] = diff_analysis_df['Categoria'].str.rsplit('.', n=1, expand=True).iloc[:, 0]

    # Se a categoria for 'MY.2022', rsplit('.', n=1) dará 'MY' e '2022'.
    # Se for 'Vehicle.Type.0', dará 'Vehicle.Type' e '0'.
    # Parece que pegar a primeira parte após rsplit('.', n=1) é o que você precisa para o nome da variável original.

    print("\nDataFrame com nome original da variável adicionado:")
    print(diff_analysis_df.head().to_string()) # Exibir as primeiras linhas para verificar

    # --- Contar a frequência das variáveis na lista ordenada por diferença ---

    print("\nContagem das Variáveis nas Categorias com Maiores Diferenças:")

    # Você quer contar a frequência das variáveis que aparecem nas categorias
    # com maior distância. Podemos simplesmente contar as ocorrências de 'Nome_Variavel_Original'
    # no DataFrame já ordenado. As variáveis no topo da contagem correspondem
    # àquelas cujas categorias estão entre as que mais se moveram.

    variable_difference_counts = diff_analysis_df['Nome_Variavel_Original'].value_counts()

    print(variable_difference_counts)

    print("\n--- Análise da Contagem ---")
    print("As variáveis com maior contagem na lista acima são aquelas cujas categorias estão mais frequentemente entre as que apresentaram maiores diferenças de posição no mapa MCA entre os dados Real e Sintético.")
    print("Isso sugere que a estrutura de associação dessas variáveis específicas pode não ter sido tão bem replicada quanto a de outras variáveis.")


else:
    print("Erro: O DataFrame 'table_of_differences' não foi encontrado. Certifique-se de executar os blocos anteriores.")

In [ ]:
variable_difference_counts

In [ ]:
quality_details = quality_report.get_details(property_name='Column Pair Trends')

In [ ]:
quality_details.to_csv('quality_details.csv', index=False)

In [ ]:
quality_report.get_info()

In [ ]:
quality_report.get_properties()

In [ ]:
diagnostic.get_properties()

In [ ]:
fig=diagnostic.get_visualization(property_name='Data Validity')
fig.show()

In [ ]:
diagnostic_details = diagnostic.get_details(property_name='Data Validity')

In [ ]:
diagnostic_details.to_csv('diagnostic_details.csv', index=False)

In [ ]:
diagnostic.get_info()

In [ ]:
diagnostic.report_info

In [ ]:
# 3. plot the data
fig = get_column_plot(
    real_data=processed_df,
    synthetic_data=synthetic_data,
    metadata=metadata,
    column_name='Warranty'
)

fig.show()

In [ ]:
from sdmetrics.reports.single_table import DiagnosticReport

report = DiagnosticReport()

In [ ]:
df.columns

In [ ]:
synthetic_data.to_csv('alignment_synthetic_data.csv', index=False)

In [ ]:
synthetic_data.Warranty.value_counts(normalize=True)

In [ ]:
synthesizer.get_parameters()

In [ ]:
synthetic_data.isnull().sum()